In [8]:
# instatiate embedder
import bend

nt_embedder = bend.embedders.NucleotideTransformerEmbedder('InstaDeepAI/nucleotide-transformer-500m-1000g')

In [2]:
# create generator to load and embed BED regions
from bend.io.sequtils import embed_from_multilabled_bed_gen, Fasta

gen = embed_from_multilabled_bed_gen('samples.bed', '../meta/chr19.fa', label_column_idx=6, label_depth=125, embedder=nt_embedder)

In [3]:
# convert to tf.data.Dataset
from bioio.tf import dataset_from_iterable

dataset = dataset_from_iterable(gen)
dataset.element_spec

0it [00:00, ?it/s]

{'inputs': TensorSpec(shape=(None, None), dtype=tf.float32, name=None),
 'outputs': TensorSpec(shape=(None,), dtype=tf.int64, name=None)}

In [4]:
# write dataset to tfrecord (this may take a while, sequences are embedded on-the-fly)
from bioio.tf import dataset_to_tfrecord

dataset_to_tfrecord(dataset, 'samples.tfrecord')

89it [00:31,  2.81it/s]


In [7]:
# create torch dataset & dataloader from tfrecord
import torch
from bend.io.datasets import TFRecordIterableDataset

ds = TFRecordIterableDataset(['samples.tfrecord'])
dl = torch.utils.data.DataLoader(ds, batch_size=8, num_workers=0)

x, y = next(iter(dl))
print(x.shape, y.shape)

torch.Size([8, 100, 1280]) torch.Size([8, 125])
